# Text-To-Speech (TTS)

In [1]:
# !huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5/

In [22]:
import os
import pandas as pd
from IPython.display import Audio, display
from audioseal import AudioSeal
import librosa
import torch
from tqdm import tqdm

In [2]:
# Load transcriptions
transcriptions = pd.read_csv("../Dataset/Transcriptions/transcriptions_complete.csv")
transcriptions = transcriptions.sort_values("confidence", ascending=False).reset_index(drop = True)

In [ ]:
# Import the detector
detector = AudioSeal.load_detector("audioseal_detector_16bits")

In [ ]:
# Parameters
watermark_path = "../Dataset/Watermarked Audio"
target_sr = 16000
device = "cpu"

# Save results
results_unwater_audioseal = []


for i in tqdm(len(transcriptions)):
    # Find the name of the clip
    clip_path = transcriptions.loc[i, "clip"]
    # Retrieve the transcription of the clip
    transcript_i = transcriptions.loc[i, "transcription"]
    # Find the watermarked clip associated with the original clip
    clip_water_path = [i for i in os.listdir(watermark_path) if clip_path[:-4] in i][0]

    # Import the watermarked audio
    # audio_water = Audio(filename= os.path.join(watermark_path, clip_water_path))

    # Detect the probability of watermark
    # Load the watermarked audio file
    wav_watermarked, sr = librosa.load(os.path.join(watermark_path, clip_water_path), sr = target_sr)
    # Convert to a PyTorch tensor
    wav_watermarked_tensor = torch.tensor(wav_watermarked).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, samples)
    # Detect the watermark
    result_water, _ = detector.detect_watermark(wav_watermarked_tensor, sr)

    # Clone the audio
    # Enter the path to the audio file here
    src_audio = os.path.join(watermark_path, clip_water_path)

    !python fish-speech-main/fish_speech/models/vqgan/inference.py \
        -i "{src_audio}" \
        --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth" \
        --device "{device}"

    # Change the name of the file
    clip_unwater_path = clip_water_path.replace("watermarked", "unwatermarked")
    os.rename("fake.wav", clip_unwater_path)

    # Find the watermark
    wav_unwater, sr = librosa.load(clip_unwater_path, sr = target_sr)
    # Convert to a PyTorch tensor
    wav_unwater_tensor = torch.tensor(wav_unwater).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, samples)
    # Detect the watermark
    result_unwater, _ = detector.detect_watermark(wav_unwater_tensor, sr)

    results_unwater_audioseal.append([clip_water_path, result_water, clip_unwater_path, result_unwater])

In [68]:
!cd "/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/fish-speech-main" && \
export PYTHONPATH="${PYTHONPATH}:/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/fish-speech-main" && \
    python fish_speech/models/text2semantic/inference.py \
    --text "{transcript_i}" \
    --prompt-text "{transcript_i}" \
    --prompt-tokens "/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/fake.npy" \
    --checkpoint-path "/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/checkpoints/fish-speech-1.5" \
    --num-samples 2 \
    --device "cpu"
    # --compile

2025-03-16 21:32:06.471 | INFO     | __main__:main:1056 - Loading model ...
2025-03-16 21:32:14.409 | INFO     | __main__:load_model:681 - Restored model from checkpoint
2025-03-16 21:32:14.409 | INFO     | __main__:load_model:687 - Using DualARTransformer
2025-03-16 21:32:14.415 | INFO     | __main__:main:1070 - Time to load model: 7.94 seconds
2025-03-16 21:32:14.430 | INFO     | __main__:generate_long:788 - Encoded text: {transcript_i}
2025-03-16 21:32:14.430 | INFO     | __main__:generate_long:806 - Generating sentence 1/1 of sample 1/2
  0%|                                         | 24/8055 [00:03<16:51,  7.94it/s]
2025-03-16 21:32:21.347 | INFO     | __main__:generate_long:860 - Generated 26 tokens in 6.92 seconds, 3.76 tokens/sec
2025-03-16 21:32:21.347 | INFO     | __main__:generate_long:863 - Bandwidth achieved: 2.40 GB/s
2025-03-16 21:32:21.348 | INFO     | __main__:main:1103 - Sampled text: {transcript_i}
2025-03-16 21:32:21.348 | INFO     | __main__:main:1108 - Saved codes 

In [76]:
!cd "/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/fish-speech-main" && \
python fish_speech/models/vqgan/inference.py \
    -i '/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/fish-speech-main/temp/codes_1.npy' \
    --checkpoint-path '/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth' \
    --device "cpu"

19510.77s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/opt/anaconda3/envs/VocalDiffusionAttack/lib/python3.12/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/opt/anaconda3/envs/VocalDiffusionAttack/lib/python3.12/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/opt/anaconda3/envs/VocalDiffusionAttack/lib/python3.12/site-packages/vector_quantize_pytorch/finite_scalar_quantization.py:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/opt/anaconda3/envs/VocalDiffusionAttack/lib/python3.12/site-packages/vector_quantize_pytorch/lookup_free_quantization.py:209: FutureWarning: `torch.cuda.amp.autoc

In [77]:
from IPython.display import Audio, display
audio = Audio(filename='/Users/lucas/Library/CloudStorage/OneDrive-UniversityofWaterloo/2025-W/CS858 - Security & Privacy in Machine Learning/Project/Code/fish-speech-main/fake.wav')
display(audio)